In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
from functools import partial
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb

from source.quantization import quantize_tensor

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda111.so


/trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/.singularity.d/libs did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/.singularity.d/env/91-environment.sh')}
  warn(msg)
/trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following dir

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /trinity/home/daria.cherniuk/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


In [3]:
def find_all_linear_names(bits, model):
    cls = bnb.nn.Linear4bit if bits == 4 else (bnb.nn.Linear8bitLt if bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model_id = "huggyllama/llama-7b"
cache_dir='/gpfs/gpfs0/daria.cherniuk/huggycache/'
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             # quantization_config=bnb_config, 
                                             device_map={"":0},
                                             cache_dir=cache_dir)

Loading checkpoint shards: 100%|██████████| 2/2 [00:48<00:00, 24.05s/it]


In [6]:
model.device

device(type='cuda', index=0)

In [7]:
!nvidia-smi

Sat Aug  5 00:29:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:27:00.0 Off |                    0 |
| N/A   37C    P0    70W / 400W |  26615MiB / 81920MiB |      1%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
state_dict = model.state_dict()

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [12]:
i = 0
f'model.layers.{i}.self_attn.q_proj.weight' in state_dict, \
f'model.layers.{i}.self_attn.k_proj.weight' in state_dict, \
f'model.layers.{i}.self_attn.v_proj.weight' in state_dict, \
f'model.layers.{i}.self_attn.o_proj.weight' in state_dict, \
f'model.layers.{i}.mlp.up_proj.weight' in state_dict, \
f'model.layers.{i}.mlp.gate_proj.weight' in state_dict, \
f'model.layers.{i}.mlp.down_proj.weight' in state_dict,

(True, True, True, True, True, True, True)

In [33]:
model.model.layers[0].self_attn.q_proj.weight.shape

torch.Size([4096, 4096])

In [50]:
# modules = find_all_linear_names(bits=4, model=model)
modules = find_all_linear_names(bits=32, model=model)
modules

['q_proj', 'o_proj', 'up_proj', 'v_proj', 'gate_proj', 'down_proj', 'k_proj']

In [51]:
# in_features = model.model.layers[0].self_attn.q_proj.in_features
# out_features = model.model.layers[0].self_attn.q_proj.out_features
# W = state_dict['model.layers.0.self_attn.q_proj.weight'].reshape(out_features, in_features)
W = state_dict['model.layers.0.self_attn.q_proj.weight']
W.shape

torch.Size([4096, 4096])

In [52]:
def project_rank(H, rank):
    U, S, Vt = torch.linalg.svd(H)
    return U[:,:rank] @ torch.diag(S[:rank]) @ Vt[:rank]

def admm_iteration(H, U, W, H2, proj_func, rho=1.0, max_iter=50, eps=1e-8):
    # rank = H.shape[1]
    # rho = torch.trace(G) / rank
    # rho = 1.0
    for j in range(1, max_iter):
        H_ = (rho*(H + U) + W - H2) / (1 + rho)
        H_prev = H.clone()
        # H = quantize_tensor(H_T - U, qscheme=qscheme, bits=bits)
        H = proj_func(H_ - U)
        U += H - H_

        r = torch.sum((H - H_)**2) / torch.sum(H**2)
        s = torch.sum((H - H_prev)**2) / torch.sum(U**2) 
        if r < eps and s < eps: 
            break
            
    return H, U

In [53]:
%%time
# projection functions
bits = 4
rank = 4
init = 'random'
quantize_func = partial(quantize_tensor, qscheme='tensor_minmax', bits=bits)
project_func = partial(project_rank, rank=rank)

# original matrix
W = state_dict['model.layers.0.self_attn.q_proj.weight']

# init W_q as quantized W or as random matrix
if init == 'random':
    W_q = torch.randn(*W.shape, device=device)
else:
    W_q = quantize_func(W)
U_q = torch.zeros_like(W_q, device=device)

# init W_r as low-rank random matrix
W_r = torch.randn(*W.shape, device=device)
W_r = project_func(W_r)
U_r = torch.zeros_like(W_r, device=device)

print(f'Bits: {bits}, Rank: {rank}')
print(f'Diff between W and quantized W abs: {torch.linalg.norm(W - quantize_func(W)):.4f}, rel: {torch.linalg.norm(W - quantize_func(W))/torch.linalg.norm(W):.4f}')
print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')

# ADMM iteration
rho = 1.0
for i in range(500):
    W_q, U_q = admm_iteration(W_q, U_q, W, W_r, quantize_func, rho=rho)
    W_r, U_r = admm_iteration(W_r, U_r, W, W_q, project_func, rho=rho)
    if i % 10 == 0:
        print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')


Bits: 4, Rank: 4
Diff between W and quantized W abs: 122.0862, rel: 0.9553
Diff between W and (W_q + W_r) abs: 4106.7480, rel: 32.1340
Diff between W and (W_q + W_r) abs: 129.7543, rel: 1.0153
Diff between W and (W_q + W_r) abs: 122.7363, rel: 0.9604
Diff between W and (W_q + W_r) abs: 123.5316, rel: 0.9666
Diff between W and (W_q + W_r) abs: 120.9213, rel: 0.9462


KeyboardInterrupt: 

In [55]:
%%time
# projection functions
bits = 4
rank = 8
init = 'random'
quantize_func = partial(quantize_tensor, qscheme='tensor_minmax', bits=bits)
project_func = partial(project_rank, rank=rank)

# original matrix
W = state_dict['model.layers.0.self_attn.q_proj.weight']

# init W_q as quantized W or as random matrix
if init == 'random':
    W_q = torch.randn(*W.shape, device=device)
else:
    W_q = quantize_func(W)
U_q = torch.zeros_like(W_q, device=device)

# init W_r as low-rank random matrix
W_r = torch.randn(*W.shape, device=device)
W_r = project_func(W_r)
U_r = torch.zeros_like(W_r, device=device)

print(f'Bits: {bits}, Rank: {rank}')
print(f'Diff between W and quantized W abs: {torch.linalg.norm(W - quantize_func(W)):.4f}, rel: {torch.linalg.norm(W - quantize_func(W))/torch.linalg.norm(W):.4f}')
print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')

# ADMM iteration
rho = 1.0
for i in range(100):
    W_q, U_q = admm_iteration(W_q, U_q, W, W_r, quantize_func, rho=rho)
    W_r, U_r = admm_iteration(W_r, U_r, W, W_q, project_func, rho=rho)
    if i % 10 == 0:
        print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')


Bits: 4, Rank: 8
Diff between W and quantized W abs: 122.0862, rel: 0.9553
Diff between W and (W_q + W_r) abs: 4114.1504, rel: 32.1919
Diff between W and (W_q + W_r) abs: 131.3533, rel: 1.0278
Diff between W and (W_q + W_r) abs: 131.2432, rel: 1.0269
Diff between W and (W_q + W_r) abs: 131.4433, rel: 1.0285
Diff between W and (W_q + W_r) abs: 131.2081, rel: 1.0267
Diff between W and (W_q + W_r) abs: 130.6566, rel: 1.0223
Diff between W and (W_q + W_r) abs: 130.7635, rel: 1.0232
Diff between W and (W_q + W_r) abs: 129.7207, rel: 1.0150
Diff between W and (W_q + W_r) abs: 129.1010, rel: 1.0102
Diff between W and (W_q + W_r) abs: 128.7581, rel: 1.0075
Diff between W and (W_q + W_r) abs: 128.8057, rel: 1.0079
CPU times: user 2h 32min 12s, sys: 167 ms, total: 2h 32min 12s
Wall time: 2h 32min 18s


In [57]:
%%time
# projection functions
bits = 2
rank = 4
init = 'random'
quantize_func = partial(quantize_tensor, qscheme='tensor_minmax', bits=bits)
project_func = partial(project_rank, rank=rank)

# original matrix
W = state_dict['model.layers.0.self_attn.q_proj.weight']

# init W_q as quantized W or as random matrix
if init == 'random':
    W_q = torch.randn(*W.shape, device=device)
else:
    W_q = quantize_func(W)
U_q = torch.zeros_like(W_q, device=device)

# init W_r as low-rank random matrix
W_r = torch.randn(*W.shape, device=device)
W_r = project_func(W_r)
U_r = torch.zeros_like(W_r, device=device)

print(f'Bits: {bits}, Rank: {rank}')
print(f'Diff between W and quantized W abs: {torch.linalg.norm(W - quantize_func(W)):.4f}, rel: {torch.linalg.norm(W - quantize_func(W))/torch.linalg.norm(W):.4f}')
print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')

# ADMM iteration
rho = 1.0
for i in range(100):
    W_q, U_q = admm_iteration(W_q, U_q, W, W_r, quantize_func, rho=rho)
    W_r, U_r = admm_iteration(W_r, U_r, W, W_q, project_func, rho=rho)
    if i % 10 == 0:
        print(f'Diff between W and (W_q + W_r) abs: {torch.linalg.norm(W - W_r - W_q):.4f}, rel: {torch.linalg.norm(W - W_r - W_q)/torch.linalg.norm(W):.4f}')


Bits: 2, Rank: 4
Diff between W and quantized W abs: 929.1003, rel: 7.2699
Diff between W and (W_q + W_r) abs: 4106.0610, rel: 32.1286
Diff between W and (W_q + W_r) abs: 705.8522, rel: 5.5231
Diff between W and (W_q + W_r) abs: 503.7399, rel: 3.9416
Diff between W and (W_q + W_r) abs: 495.2279, rel: 3.8750
Diff between W and (W_q + W_r) abs: 765.5813, rel: 5.9904
Diff between W and (W_q + W_r) abs: 973.9545, rel: 7.6209
Diff between W and (W_q + W_r) abs: 1115.0795, rel: 8.7251
Diff between W and (W_q + W_r) abs: 1219.1031, rel: 9.5391
Diff between W and (W_q + W_r) abs: 1297.3093, rel: 10.1510
Diff between W and (W_q + W_r) abs: 1357.7460, rel: 10.6239
Diff between W and (W_q + W_r) abs: 1412.9738, rel: 11.0561
CPU times: user 2h 45min 46s, sys: 1.24 s, total: 2h 45min 47s
Wall time: 2h 46min 7s
